In [330]:
import pandas as pd
import numpy_financial as npf
import numpy as np

In [331]:
#key target characteristics
ARR = 12000
GAAPR_recurring = 12000
GAAPR_nonrecurring = 6500
GAAP_totalrevenue = GAAPR_recurring+GAAPR_nonrecurring
EBITDA = 4500
EBITDA_margin = EBITDA/GAAP_totalrevenue
ARR_multiple = 8
Price = 96000
dcf_discountrate = 0.15
tvgrowthrate = 0.045

#base case performance assumptions
ARR_growth = 0.2
Nonrecurring_revenuegrowth = 0.03
EBITDA_marginincrease = 0.05
num_years = 6 # six to account for year 0/close
#EBITDA_exitvalmultiple =
ARR_exitvalmultiple = ARR_multiple

#desired financing
senior_debt = 0
mezz_financing = 1.25*GAAPR_nonrecurring/Price
equity = 1 - senior_debt - mezz_financing

#TODO class specific stuff to be implemented later

#Detailed Assumptions
ida = 0
secured_debt_interestrate = 0.07
mezz_interest_rate = 0
corp_income_taxrate = 0.21
annual_capex = 0
seller_financing = 15000
PIK = 0.12

In [332]:
df = pd.DataFrame({'Year': range(num_years)})

PROFIT & LOSS TITLE?

In [333]:
df['ARR'] = ARR * (1+ARR_growth)**df['Year']

In [334]:
df['ARR Growth'] = np.full(num_years,ARR_growth)
df.at[0, 'ARR Growth'] = np.nan

In [335]:
df['GAAP Recurring Revenue'] = None
df.at[0, 'GAAP Recurring Revenue'] = ARR
for i in range(1, num_years):
    df.at[i, 'GAAP Recurring Revenue'] = df.at[i, 'ARR'] * df.at[i - 1, 'GAAP Recurring Revenue'] / df.at[i - 1, 'ARR']


In [336]:
df['Non-Recurring Revenue'] = GAAPR_nonrecurring * (1+Nonrecurring_revenuegrowth)**df['Year']

In [337]:
df['Non-Recurring Revenue Growth'] = np.full(num_years,Nonrecurring_revenuegrowth)
df.at[0, 'Non-Recurring Revenue Growth'] = np.nan

In [338]:
df['Total Revenue'] = df['GAAP Recurring Revenue']+df['Non-Recurring Revenue']

In [339]:
df['Total Revenue Growth'] = [np.nan, 0.14, 0.15, 0.15, 0.16, 0.16]

In [340]:
df['EBITDA Margin'] = None
df.at[0,'EBITDA Margin'] = EBITDA_margin
for i in range(1, num_years):
    df.at[i, 'EBITDA Margin'] =df.at[i-1,'EBITDA Margin']+EBITDA_marginincrease

In [341]:
df['EBITDA'] = df['Total Revenue']*df['EBITDA Margin']

In [342]:
df['Depreciation and Amortization'] = None
df.at[0,'Depreciation and Amortization'] = ida
for i in range(1, num_years):
    df.at[i,'Depreciation and Amortization'] = (df.at[i-1,'Depreciation and Amortization']/df.at[i-1,'ARR'])*df.at[i, 'ARR']

SHIT BELONGS HERE FOR THE INTEREST EXPENSE

In [343]:
df['Interest Expense'] = 0

In [344]:
df['Earnings Before Taxes'] = df['EBITDA']-df['Depreciation and Amortization']-df['Interest Expense']

In [345]:
df['Corporate Income Tax Rate'] = np.full(num_years,corp_income_taxrate)

In [346]:
df['Corporate Income Tax'] = None
for i in range(num_years):
    if i == 0:
        df.at[i, 'Corporate Income Tax'] = None  # Handle the first value separately
    else:
        df.at[i, 'Corporate Income Tax'] = df.at[i, 'Earnings Before Taxes'] * df.at[i, 'Corporate Income Tax Rate']

In [347]:
df['Net Income'] = None
for i in range(num_years):
    if i == 0:
        df.at[i, 'Net Income'] = None
    else:
        df.at[i, 'Net Income'] = df.at[i,'Earnings Before Taxes']-df.at[i, 'Corporate Income Tax']

CASH FLOW TITLE?

In [348]:
df['CF_EBITDA'] = df['EBITDA']
df.at[0,'CF_EBITDA']= np.nan

In [349]:
df['Cash Taxes'] = -1*df['Corporate Income Tax']

In [350]:
df['Changes In NWC'] = None
for i in range(1,num_years):
    if i == 0:
        df.at[i,'Changes in NWC'] = np.nan
    df.at[i,'Changes In NWC'] = ((df.at[i,'ARR']-df.at[i-1,'ARR'])*0.2)*-1

In [351]:
df['Capex'] = np.full(num_years,annual_capex)

In [352]:
df['Free Cash Flow To Firm'] = df['CF_EBITDA']+df['Cash Taxes']+df['Changes In NWC']+df['Capex']

In [353]:
df['Cash Interest Expense'] = -1*df['Interest Expense']

In [354]:
df['Mandatory Debt Paydown'] = 0

In [355]:
df['Cash Flow For Voluntary Debt Paydown'] = df['Free Cash Flow To Firm']+df['Cash Interest Expense']+df['Mandatory Debt Paydown']
df

,Year,ARR,ARR Growth,GAAP Recurring Revenue,Non-Recurring Revenue,Non-Recurring Revenue Growth,Total Revenue,Total Revenue Growth,EBITDA Margin,EBITDA,...,Corporate Income Tax,Net Income,CF_EBITDA,Cash Taxes,Changes In NWC,Capex,Free Cash Flow To Firm,Cash Interest Expense,Mandatory Debt Paydown,Cash Flow For Voluntary Debt Paydown
0,0,12000.00,NaN,12000,6500.000000,NaN,18500.0,NaN,0.243243,4500.0,...,None,None,NaN,NaN,None,0,NaN,0,0,NaN
1,1,14400.00,0.2,14400.0,6695.000000,0.03,21095.0,0.14,0.293243,6185.966216,...,1299.052905,4886.913311,6185.966216,-1299.052905,-480.0,0,4406.913311,0,0,4406.913311
2,2,17280.00,0.2,17280.0,6895.850000,0.03,24175.85,0.15,0.343243,8298.197162,...,1742.621404,6555.575758,8298.197162,-1742.621404,-576.0,0,5979.575758,0,0,5979.575758
3,3,20736.00,0.2,20736.0,7102.725500,0.03,27838.7255,0.15,0.393243,10947.390703,...,2298.952048,8648.438656,10947.390703,-2298.952048,-691.2,0,7957.238656,0,0,7957.238656
4,4,24883.20,0.2,24883.2,7315.807265,0.03,32199.007265,0.16,0.443243,14271.992409,...,2997.118406,11274.874003,14271.992409,-2997.118406,-829.44,0,10445.434003,0,0,10445.434003
5,5,29859.84,0.2,29859.84,7535.281483,0.03,37395.121483,0.16,0.493243,18444.891002,...,3873.42711,14571.463891,18444.891002,-3873.42711,-995.328,0,13576.135891,0,0,13576.135891


In [356]:
df['Senior Debt'] = 0

In [357]:
df['Voluntary Debt Paydown of Senior Debt'] = None
for i in range(1, num_years):
    if df.at[i, 'Senior Debt'] > 0:
        paydown = -1 * min(df.at[i, 'Senior Debt'] + df.at[i, 'Mandatory Debt Paydown'], df.at[i, 'Cash Flow For Voluntary Debt Paydown'])
        df.at[i, 'Voluntary Debt Paydown of Senior Debt'] = paydown
    else:
        df.at[i, 'Voluntary Debt Paydown of Senior Debt'] = 0

In [358]:
df['Seller Financing'] = None
df['Voluntary Debt Paydown of Seller Financing'] = 0
df.at[0,'Seller Financing'] = seller_financing
for i in range(1, num_years):
    df.at[i,'Seller Financing'] = df.at[i-1,'Seller Financing']*(1+PIK) + df.at[i,'Voluntary Debt Paydown of Seller Financing']



In [359]:
df['Voluntary Debt Paydown of Seller Financing'] = np.where(df['Seller Financing'] > 0, -1 * np.minimum(df['Cash Flow For Voluntary Debt Paydown'] + df['Voluntary Debt Paydown of Senior Debt'], df['Seller Financing'] * (1 + PIK)),0)
df


C:\Users\chris_pedretti\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Year,ARR,ARR Growth,GAAP Recurring Revenue,Non-Recurring Revenue,Non-Recurring Revenue Growth,Total Revenue,Total Revenue Growth,EBITDA Margin,EBITDA,...,Changes In NWC,Capex,Free Cash Flow To Firm,Cash Interest Expense,Mandatory Debt Paydown,Cash Flow For Voluntary Debt Paydown,Senior Debt,Voluntary Debt Paydown of Senior Debt,Seller Financing,Voluntary Debt Paydown of Seller Financing
0,0,12000.00,NaN,12000,6500.000000,NaN,18500.0,NaN,0.243243,4500.0,...,None,0,NaN,0,0,NaN,0,None,15000,-16800.0
1,1,14400.00,0.2,14400.0,6695.000000,0.03,21095.0,0.14,0.293243,6185.966216,...,-480.0,0,4406.913311,0,0,4406.913311,0,0,16800.0,-4406.913311
2,2,17280.00,0.2,17280.0,6895.850000,0.03,24175.85,0.15,0.343243,8298.197162,...,-576.0,0,5979.575758,0,0,5979.575758,0,0,18816.0,-5979.575758
3,3,20736.00,0.2,20736.0,7102.725500,0.03,27838.7255,0.15,0.393243,10947.390703,...,-691.2,0,7957.238656,0,0,7957.238656,0,0,21073.92,-7957.238656
4,4,24883.20,0.2,24883.2,7315.807265,0.03,32199.007265,0.16,0.443243,14271.992409,...,-829.44,0,10445.434003,0,0,10445.434003,0,0,23602.7904,-10445.434003
5,5,29859.84,0.2,29859.84,7535.281483,0.03,37395.121483,0.16,0.493243,18444.891002,...,-995.328,0,13576.135891,0,0,13576.135891,0,0,26435.125248,-13576.135891
